<a href="https://colab.research.google.com/github/Vurimindi2021/CNN-to-Detect-Atelectasis-in-Chest-X-Ray/blob/main/src/image_test_train3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import math
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras import layers, Model, optimizers, callbacks
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tensorflow.keras.metrics import BinaryAccuracy, Recall, Precision, SensitivityAtSpecificity
from tensorflow.keras.metrics import FalseNegatives, FalsePositives
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.preprocessing.image import image, load_img, img_to_array
from numpy import expand_dims
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.utils import plot_model

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#Paths 
train_path = '/content/drive/MyDrive/output/train'
valid_path = '/content/drive/MyDrive/output/val'
test_path  = '/content/drive/MyDrive/output/test'
chkpt_path = '/content/drive/MyDrive/check_point1'
log_path = os.path.join("/content/drive/MyDrive/logs/fit/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
#Global Parameters
types       = ['Atelectasis','NoFinding']
batch_size  = 32
epochs      = 50

In [ ]:
#ImageDataGenerator
train_images = ImageDataGenerator(rotation_range = 5, 
                                  shear_range = 0.02,
                                  zoom_range = 0.02, 
                                  samplewise_center=True, 
                                  samplewise_std_normalization= True
                                  ).flow_from_directory(train_path, 
                                                        target_size=(256,256),
                                                        color_mode='grayscale',
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')                               
valid_images = ImageDataGenerator(rotation_range = 5, 
                                  shear_range = 0.02,
                                  zoom_range = 0.02, 
                                  samplewise_center=True, 
                                  samplewise_std_normalization= True
                                  ).flow_from_directory(valid_path, 
                                                        target_size=(256,256),
                                                        color_mode='grayscale',
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')  
test_images = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(256,256),
                                                        color_mode='grayscale',
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')

In [ ]:
train_images.next()

In [ ]:
p = train_images.next()
plt.imshow(p[0][10][:,:,0], cmap='gray')
plt.show()

In [ ]:
#Design a convolutional neural network with at least two convolution layers, at least one max-pooling layer, and at least one dropout layer
model = Sequential()
# adding layers 
# block 1
model.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
# block 2
model.add(Conv2D(96, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# block 3
model.add(Conv2D(192, kernel_size=3, activation='relu'))
model.add(Conv2D(192, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# block 4
model.add(Conv2D(384, kernel_size=3, activation='relu'))
model.add(Conv2D(384, kernel_size=3, activation='relu'))
model.add(Conv2D(384, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# block 5
model.add(Conv2D(512, kernel_size=3, activation='relu'))
model.add(Conv2D(512, kernel_size=3, activation='relu'))
model.add(Conv2D(512, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# block 5
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
# compiling the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=[BinaryAccuracy(), 
                       Recall(), 
                       Precision()
                      ])

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='/content/drive/MyDrive/figs/greyscale_sequential.png')

In [ ]:
train_size  = 3919
val_size    = 840
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
steps_per_epoch = compute_steps_per_epoch(train_size)
val_steps = compute_steps_per_epoch(val_size)

In [ ]:
checkpoint = callbacks.ModelCheckpoint(filepath=chkpt_path,
                                    save_weights_only=True,
                                    monitor='val_binary_accuracy',
                                    mode='max',
                                    save_best_only=True)
earlystop = callbacks.EarlyStopping(monitor='val_binary_accuracy', 
                                    min_delta=0.001, 
                                    patience=5, 
                                    verbose=0,
                                    mode='auto', 
                                    baseline=None, 
                                    restore_best_weights=True)
tensorboard = callbacks.TensorBoard(log_dir=log_path, 
                                    histogram_freq=1,
                                    write_graph=True)

# train the model
model.fit(train_images,
          shuffle=True,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          validation_steps=val_steps,
          validation_data=valid_images,
          callbacks=[checkpoint, earlystop, tensorboard])

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(chkpt_path)
# Calculate the class probabilities
probabilities = model.predict(test_images)


In [ ]:
evaluate = model.evaluate(test_images)

In [ ]:
classes = probabilities> 0.5
classes

In [ ]:
for i in range(len(probabilities)):
	print("X=%s, Predicted=%s" % (probabilities[i], probabilities[i]))

In [ ]:
# ROC AUC Scores
roc_auc_score(y_test['label'], probabilities)

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard
import tensorflow as tf
import datetime
%tensorboard --logdir /content/drive/MyDrive/logs

In [ ]:
#ImageDataGenerator
train_images = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization= True,
                                  rotation_range = 5,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  fill_mode="nearest",
                                  cval=0.0).flow_from_directory(train_path, 
                                                        target_size=(224,224),
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')                               
valid_images = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization= True,
                                  rotation_range = 5,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  fill_mode="nearest",
                                  cval=0.0).flow_from_directory(valid_path, 
                                                        target_size=(224,224),
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')  
test_images = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224),
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')

In [ ]:
# load model
model = VGG16(weights='imagenet', include_top=True)
model.trainable = False

In [ ]:
# adding layers 
x = model.output
# add a 2D global average pooling layer
x = Dense(1024, activation='relu')(x)
# I add a regularizing layer
x = Dropout(0.5)(x)
# add a layer for binary classification.
predictions = Dense(1, activation='sigmoid')(x)

model_vvg_random = Model(inputs=model.input, outputs=predictions)

In [ ]:
# compile the model and start training using the previously defined generators
# I keep the optimizer and loss function constant for consistency
model_vvg_random.compile(optimizer='adam', 
                         loss='binary_crossentropy',               
                         metrics=[
                                  BinaryAccuracy(), 
                                  Recall(), 
                                  Precision()
                                  ])
# summarize the model
model.summary()

In [ ]:
checkpoint = callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/check_point2',
                                    save_weights_only=True,
                                    monitor='val_binary_accuracy',
                                    mode='max',
                                    save_best_only=True)
earlystop = callbacks.EarlyStopping(monitor='val_binary_accuracy', 
                                    min_delta=0.001, 
                                    patience=5, 
                                    verbose=0,
                                    mode='auto', 
                                    baseline=None, 
                                    restore_best_weights=True)
tensorboard = callbacks.TensorBoard(log_dir=log_path, 
                                    histogram_freq=1,
                                    write_graph=True)
# The model weights (that are considered the best) are loaded into the model.
model_vvg_random.load_weights('/content/drive/MyDrive/check_point1')
# train the model
model_vvg_random.fit(train_images,
                      shuffle=True,
                      epochs=epochs,
                      steps_per_epoch=steps_per_epoch,
                      validation_steps=val_steps,
                      validation_data=valid_images,
                      callbacks=[checkpoint, earlystop, tensorboard])

In [ ]:
# Calculate the class probabilities
probabilities = model_vvg_random.predict(test_images)
classes = probabilities> 0.5
len(classes)